In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
class VegetableClassifier(nn.Module):
    def __init__(self, num_classes):
        super(VegetableClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)

        return x

In [3]:
# Define the transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Define custom Albumentations transformations
augmentations = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    # Add more augmentations as needed
    A.Normalize(),
    ToTensorV2(),
])

In [13]:
# Define custom dataset class with Albumentations augmentations
class AlbumentationsDataset(DatasetFolder):
    def __init__(self, root, augmentations=None, **kwargs):
        super(AlbumentationsDataset, self).__init__(root, **kwargs)
        self.augmentations = augmentations

    def __getitem__(self, index):
        path, target = self.samples[index]
        image = self.loader(path)

        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']

        if self.transform is not None:
            image = self.transform(image)

        return image, target

NameError: name 'DatasetFolder' is not defined

In [5]:
# Initialize the dataset loaders
train_dataset = ImageFolder("C:/Users/8092/Placement Assesment/Computer Vision/Vegetable Images/train", 
                            transform=transforms.ToTensor())
test_dataset = ImageFolder("C:/Users/8092/Placement Assesment/Computer Vision/Vegetable Images/test", 
                           transform=transform)
validation_dataset = ImageFolder("C:/Users/8092/Placement Assesment/Computer Vision/Vegetable Images/validation", 
                                 transform=transform)

In [6]:
# Apply augmentations to the training dataset
train_dataset.transform = augmentations

In [7]:
# Define the model and optimizer
model = VegetableClassifier(num_classes=len(train_dataset.classes))
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [8]:
# Enable distributed parallel training if available
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

In [9]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

VegetableClassifier(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU(inplace=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=512, bias=True)
  (relu4): ReLU(inplace=True)
  (fc2): Linear(in_features=512, out_features=15, bias=True)
)

In [10]:
# Initialize TensorBoard writer
writer = SummaryWriter()

In [11]:
# Training loop
num_epochs = 10
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

In [12]:

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_samples = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        train_loss += loss.item() * images.size(0)
        total_samples += images.size(0)

    # Calculate training accuracy and loss
    train_accuracy = 100.0 * train_correct / total_samples
    train_loss = train_loss / total_samples

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_samples = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\8092\Anaconda3\envs\cv\lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\8092\Anaconda3\envs\cv\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\8092\Anaconda3\envs\cv\lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "C:\Users\8092\Anaconda3\envs\cv\lib\site-packages\torchvision\datasets\folder.py", line 231, in __getitem__
    sample = self.transform(sample)
  File "C:\Users\8092\Anaconda3\envs\cv\lib\site-packages\albumentations\core\composition.py", line 188, in __call__
    raise KeyError("You have to pass data to augmentations as named arguments, for example: aug(image=image)")
KeyError: